In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import KFold
from MTS import *
from orthogonal_array import *

In [2]:
# ファイル読み込み
IN_FILE = '../data/OnlineNewsPopularity.csv'
df = pd.read_csv(IN_FILE, header=0, index_col=None, sep='\s*,\s*', engine='python')

# 基礎分析

In [3]:
df.shape

(39644, 61)

In [4]:
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [5]:
df.describe()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
count,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,...,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000
mean,354.530471,10.398749,546.514731,0.548216,0.996469,0.689175,10.883690,3.293638,4.544143,1.249874,...,0.095446,0.756728,-0.259524,-0.521944,-0.107500,0.282353,0.071425,0.341843,0.156064,3395.380184
std,214.163767,2.114037,471.107508,3.520708,5.231231,3.264816,11.332017,3.855141,8.309434,4.107855,...,0.071315,0.247786,0.127726,0.290290,0.095373,0.324247,0.265450,0.188791,0.226294,11626.950749
min,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,1.000000
25%,164.000000,9.000000,246.000000,0.470870,1.000000,0.625739,4.000000,1.000000,1.000000,0.000000,...,0.050000,0.600000,-0.328383,-0.700000,-0.125000,0.000000,0.000000,0.166667,0.000000,946.000000
50%,339.000000,10.000000,409.000000,0.539226,1.000000,0.690476,8.000000,3.000000,1.000000,0.000000,...,0.100000,0.800000,-0.253333,-0.500000,-0.100000,0.150000,0.000000,0.500000,0.000000,1400.000000
75%,542.000000,12.000000,716.000000,0.608696,1.000000,0.754630,14.000000,4.000000,4.000000,1.000000,...,0.100000,1.000000,-0.186905,-0.300000,-0.050000,0.500000,0.150000,0.500000,0.250000,2800.000000
max,731.000000,23.000000,8474.000000,701.000000,1042.000000,650.000000,304.000000,116.000000,128.000000,91.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,1.000000,843300.000000


In [6]:
(df=='?').sum(axis=0)

url                              0
timedelta                        0
n_tokens_title                   0
n_tokens_content                 0
n_unique_tokens                  0
n_non_stop_words                 0
n_non_stop_unique_tokens         0
num_hrefs                        0
num_self_hrefs                   0
num_imgs                         0
num_videos                       0
average_token_length             0
num_keywords                     0
data_channel_is_lifestyle        0
data_channel_is_entertainment    0
data_channel_is_bus              0
data_channel_is_socmed           0
data_channel_is_tech             0
data_channel_is_world            0
kw_min_min                       0
kw_max_min                       0
kw_avg_min                       0
kw_min_max                       0
kw_max_max                       0
kw_avg_max                       0
kw_min_avg                       0
kw_max_avg                       0
kw_avg_avg                       0
self_reference_min_s

In [7]:
# '?'を含む行を削除
df = df[(df=='?').sum(axis=1)==0]

In [8]:
# シャッフルとインデックス更新
df = df.sample(frac=1, random_state=0) # 行シャッフル
df = df.reset_index(drop=True) # インデックスの更新

In [9]:
# データセットの作成
target_col = 'shares'
target = 17600 # 上位1000件
del_col = [target_col, 'url']
X = df.drop(del_col, axis=1)
y = (df[target_col]==target).astype(np.int32)
print(X.shape)

(39644, 59)


# 異常検出（ホールドアウト法による評価）

In [10]:
# ホールドアウト法による精度評価
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=47)

In [ ]:
# 直交表の利用
oa = np.reshape(L64, (64, -1)) # 直交表整形
del_cols = range(X.shape[1], oa.shape[1])
oa = np.delete(oa, del_cols, axis=1) # 不要列を削除
sn_ratios = orthogonal_array_test(oa, X_train, y_train)
sn_averages_pos, sn_averages_neg = orthogonal_array_sign(oa, sn_ratios)
print(sn_averages_pos-sn_averages_neg) # 項目ごとのSN比平均差、プラスなら異常検知に関して有意な項目

[ 4.21058163 -4.11512973  3.96319221 -4.16201672 -4.27735723  3.97630438
  4.02476261 -4.00213205  4.24144222 -4.12787707  4.13324917  4.02476261
 -3.81222405  4.24144222 -4.12787707 -4.293903   -3.81222405  4.24144222
 -4.293903   -3.81222405 -4.293903    3.79729262 -3.98756042 -3.79645376
 -3.78468207 -3.65502466  3.87138064  3.91827994  3.83847047 -4.00523215
 -3.74752075  4.00704526 -3.98756042 -3.79645376 -3.99047599  3.87138064
  4.18437344  4.00704526 -3.98756042 -3.65502466  4.00704526  4.11660103
  4.10209398 -3.99081134  4.19475377  4.09975614  4.03562758 -3.91694013
 -4.01732272 -4.08515366  4.0738333   4.11660103  4.10209398  4.19475377
  0.54279219  4.11660103 -3.84354057  4.23632951  4.09023438]


In [ ]:
# 項目選択の結果をもとに異常検知
sign_cols = np.where(sn_averages_pos-sn_averages_neg>=0, 1, 0) # 直交表のテストをもとに有用な変数を選択
arr_cols = list(itertools.compress(X_train.columns, sign_cols))
X_train_ext = X_train[arr_cols]
X_test_ext = X_test[arr_cols]

# MT法による異常度計算
mts = MTS()
mts.fit(X_train_ext[y_train==0])
anomaly_score = mts.predict(X_test_ext)
(break_even_point, score), (coverage, detection) = mts.break_even_point(anomaly_score, y_test)
anomaly_score_abnormal = anomaly_score[y_test==1]

In [ ]:
# 項目選択せずに異常検知
mts = MTS()
mts.fit(X_train[y_train==0])
anomaly_score = mts.predict(X_test)
(break_even_point, score), (coverage, detection) = mts.break_even_point(anomaly_score, y_test)
anomaly_score_abnormal = anomaly_score[y_test==1]

# 異常検出（5分割交差検証による評価）

In [ ]:
# 項目選択のないMT法（単なるMahalanobis距離）交差確認
scores = []
kf = KFold(n_splits=5, random_state=0, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 項目選択せずに異常検知
    # →平均0.5867
    mts = MTS()
    mts.fit(X_train[y_train==0])
    anomaly_score = mts.predict(X_test)
    (break_even_point, score), (coverage, detection) = mts.break_even_point(anomaly_score, y_test)
    anomaly_score_abnormal = anomaly_score[y_test==1]
    scores.append(score)
print('average: {}'.format(sum(scores)/len(scores)))

In [ ]:
# MT法で交差確認
scores = []
kf = KFold(n_splits=5, random_state=0, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 直交表の利用
    oa = np.reshape(L64, (64, -1)) # 直交表整形
    del_cols = range(X.shape[1], oa.shape[1])
    oa = np.delete(oa, del_cols, axis=1) # 不要列を削除
    sn_ratios = orthogonal_array_test(oa, X_train, y_train)
    sn_averages_pos, sn_averages_neg = orthogonal_array_sign(oa, sn_ratios)
    # print(sn_averages_pos-sn_averages_neg) # 項目ごとのSN比平均差、プラスなら異常検知に関して有意な項目
    
    # 項目選択の結果をもとに異常検知
    sign_cols = np.where(sn_averages_pos-sn_averages_neg>=0, 1, 0) # 直交表のテストをもとに有用な変数を選択
    arr_cols = list(itertools.compress(X_train.columns, sign_cols))
    X_train_ext = X_train[arr_cols]
    X_test_ext = X_test[arr_cols]

    # MT法による異常度計算
    # →平均0.6751
    mts = MTS()
    mts.fit(X_train_ext[y_train==0])
    anomaly_score = mts.predict(X_test_ext)
    (break_even_point, score), (coverage, detection) = mts.break_even_point(anomaly_score, y_test)
    anomaly_score_abnormal = anomaly_score[y_test==1]
    scores.append(score)
print('average: {}'.format(sum(scores)/len(scores)))

# すべてのサンプルを使って項目選択

In [ ]:
# 直交表の利用
oa = np.reshape(L64, (64, -1)) # 直交表整形
del_cols = range(X.shape[1], oa.shape[1])
oa = np.delete(oa, del_cols, axis=1) # 不要列を削除
sn_ratios = orthogonal_array_test(oa, X, y)
sn_averages_pos, sn_averages_neg = orthogonal_array_sign(oa, sn_ratios)
print(sn_averages_pos-sn_averages_neg) # 項目ごとのSN比平均差、プラスなら異常検知に関して有意な項目

# 項目選択の結果を出力
sign_cols = np.where(sn_averages_pos-sn_averages_neg>=0, 1, 0) # 直交表のテストをもとに有用な変数を選択
arr_cols = list(itertools.compress(X.columns, sign_cols))
print(arr_cols)